In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(42)

# Distribución objetivo (A, B, C, _)
target_probs = torch.tensor([0.4, 0.3, 0.2, 0.1])

# Red neuronal simple
class SymbolGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.Softmax(dim=1)  # salida = distribución de probabilidad
        )

    def forward(self, x):
        return self.net(x)

x_train = torch.eye(4)

# Red y entrenamiento
model = SymbolGenerator()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.05)

# Entrenamiento
epochs = 75
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(x_train)
    loss = criterion(output.mean(dim=0), target_probs)
    loss.backward()
    optimizer.step()
    if epoch % 50 == 0:
        print(f"Epoch {epoch}: loss = {loss.item():.6f}")

# Resultado final
output = model(x_train)
generated_probs = output.mean(dim=0).detach().numpy()
symbols = ['A', 'B', 'C', '_']

print("\nDistribución generada:")
for s, p in zip(symbols, generated_probs):
    print(f"{s}: {p:.3f}")


Epoch 0: loss = 0.004701
Epoch 50: loss = 0.000003

Distribución generada:
A: 0.401
B: 0.299
C: 0.200
_: 0.100


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Palabras y secuencias codificadas
word_list = ["hola", "mundo", "amigo", "casa", "perro", "gato", "sol", "luna"]
word_to_bits = {
    "hola": "000", "mundo": "001", "amigo": "010", "casa": "011",
    "perro": "100", "gato": "101", "sol": "110", "luna": "111"
}

# Codificar secuencias entre 0 y 511
def seq_to_index(seq):
    bits = ''.join(word_to_bits[w] for w in seq)
    return int(bits, 2)

# Distribución objetivo sobre 512 posibles secuencias
target_probs = torch.zeros(512)
target_probs[seq_to_index(["hola", "mundo", "amigo"])] = 0.30
target_probs[seq_to_index(["amigo", "casa", "perro"])] = 0.25
target_probs[seq_to_index(["perro", "gato", "sol"])]   = 0.20
target_probs[seq_to_index(["sol", "luna", "hola"])]    = 0.25

# input dummy -> salida 512 clases
class ClassicalGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 128),
            nn.ReLU(),
            nn.Linear(128, 512),
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        return self.net(x)

# Entrenamiento
torch.manual_seed(0)
model = ClassicalGenerator()
optimizer = optim.Adam(model.parameters(), lr=0.005)
criterion = nn.MSELoss()

x_dummy = torch.ones((1, 1))

epochs = 1500
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(x_dummy)
    loss = criterion(output[0], target_probs)
    loss.backward()
    optimizer.step()
    if epoch % 200 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

output_probs = model(x_dummy).detach().numpy().flatten()
for seq, target in zip([
    ["hola", "mundo", "amigo"],
    ["amigo", "casa", "perro"],
    ["perro", "gato", "sol"],
    ["sol", "luna", "hola"]
], [0.30, 0.25, 0.20, 0.25]):
    idx = seq_to_index(seq)
    gen = output_probs[idx]
    print(f"Secuencia: {' '.join(seq):20s} | Prob. generada: {gen:.3f} | Prob. objetivo: {target:.2f}")


Epoch 0, Loss: 0.000496
Epoch 200, Loss: 0.000000
Epoch 400, Loss: 0.000000
Epoch 600, Loss: 0.000000
Epoch 800, Loss: 0.000000
Epoch 1000, Loss: 0.000000
Epoch 1200, Loss: 0.000000
Epoch 1400, Loss: 0.000000
Secuencia: hola mundo amigo     | Prob. generada: 0.300 | Prob. objetivo: 0.30
Secuencia: amigo casa perro     | Prob. generada: 0.249 | Prob. objetivo: 0.25
Secuencia: perro gato sol       | Prob. generada: 0.200 | Prob. objetivo: 0.20
Secuencia: sol luna hola        | Prob. generada: 0.250 | Prob. objetivo: 0.25


In [ ]:
!pip install torchviz graphviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from torchviz import make_dot

# Entrada ficticia
x_dummy = torch.ones((1, 1), requires_grad=True)
y = model(x_dummy)

# Visualizar grafo
dot = make_dot(y, params=dict(list(model.named_parameters())))
dot.render("red_clasica", format="png")

'red_clasica.png'